In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
cd ~/demres

/Users/zurfarosa/demres


In [13]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

from demres.common.helper_functions import *
from demres.common.process_entries import *

In [14]:
pd.options.display.max_rows = 5000
pd.set_option('display.max_columns', None)

In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [16]:
all_entries = get_all_entries()

In [7]:
prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [8]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_all_dementia_10_to_5.csv',delimiter=',')

FileNotFoundError: File b'data/pt_data/processed_data/pt_features_demins_all_dementia_10_to_5.csv' does not exist

In [ ]:
pegprod = pd.read_csv('data/dicts/proc_pegasus_prod.csv')
prescs = pd.merge(prescriptions,pegprod[['prodcode','route','substance strength','drug substance name']],how='left')

In [ ]:
prescs = pd.merge(prescs,pt_features[['patid','index_date']],how='left')

In [11]:
# pt_features[pt_features['current_smoker']==1].sample(15)

## Explore patient history

In [37]:
patid=2422

In [38]:
hist = get_patient_history(all_entries,patid).sort_values(by='eventdate')

In [40]:
hist.sort_values(by='eventdate')

,patid,medcode,prodcode,eventdate,sysdate,type,description
116,2422,11.0,NaN,1973-02-12,2000-05-11,clinical,Cervical smear: negative
70,2422,NaN,NaN,1973-02-12,2000-05-11,consultation,NaN
54,2422,NaN,NaN,1990-03-02,2000-05-11,consultation,NaN
184,2422,69.0,NaN,1990-04-05,2000-05-11,immunisation,First tetanus vaccination
121,2422,385.0,NaN,1990-04-05,2000-05-11,clinical,Drinks rarely
66,2422,NaN,NaN,1990-04-05,2000-05-11,consultation,NaN
94,2422,1.0,NaN,1990-04-05,2000-05-11,clinical,O/E - blood pressure reading
119,2422,60.0,NaN,1990-04-05,2000-05-11,clinical,Current non-smoker
55,2422,NaN,NaN,1990-05-15,2000-05-11,consultation,NaN
185,2422,75.0,NaN,1990-05-18,2000-05-11,immunisation,Second tetanus vaccination


In [22]:
pt_details = pt_features[pt_features['patid']==patid]

In [15]:
pt_details.T

""
patid
yob
pracid
female
index_date
isCase
final dementia medcode
data_start
data_end
matchid


In [16]:
# matching_pt = pt_features[(pt_features['matchid']==pt_details['matchid'].iloc[0])&(pt_features['patid']!=pt_details['patid'].iloc[0])]
# matching_pt_id = matching_pt['patid'].iloc[0]

In [17]:
# matching_pt.T

In [18]:
# matching_pt_hist = get_patient_history(all_entries,matching_pt_id).sort_values(by='eventdate')

In [18]:
index_date = pd.to_datetime(pt_details['index_date']).iloc[0]

IndexError: single positional indexer is out-of-bounds

In [20]:
# hist.sort_values(by='eventdate')

In [21]:
focused_history = hist[(hist['eventdate']>index_date - timedelta(days=3650))&(hist['eventdate']<index_date-timedelta(days=1825))].sort_values(by='eventdate')

In [16]:
focused_history

In [23]:
len(focused_history.loc[focused_history['type']=='consultation'])

276

## Explore patient prescriptions

In [24]:
prescriptions.head(1)

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq
0,57001,1989-02-08,1997-03-08,9457,807,0,4,4,50.0,2.0,0,0.0,0,0


In [25]:
prescs.sample(5)

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq,route,substance strength,drug substance name,index_date
17873851,2947183,2001-03-09,2001-03-09,377073,57,4183,33,55,1.0,8.0,0,0.0,2575,3,Oral,500mg + 8mg,Paracetamol/Codeine phosphate,NaN
16632236,1054174,2012-08-14,2012-08-14,295858,13,156174,18,35,28.0,1.0,56,0.0,921,1,Oral,50microgram,Levothyroxine sodium anhydrous,NaN
4059305,9359044,1993-03-03,1997-06-24,736946,15,7044,2623,4,100.0,0.0,30,0.0,0,0,Oral,400mg,Ibuprofen,NaN
6648032,4147075,2007-07-16,2007-07-16,358797,2017,120075,176,56,28.0,1.0,0,0.0,1,1,Oral,5mg,Zolpidem tartrate,NaN
12900427,26192134,2004-03-08,2004-03-08,1829941,91,21134,12672,180,112.0,2.0,0,0.0,1,3,Oral,20mg,Isosorbide mononitrate,NaN


In [26]:
pt_prescs = prescs[prescs['patid']==patid]

In [28]:
relev_pt_prescs = pt_prescs[(pt_prescs['eventdate']>(index_date-timedelta(days=365)*10))&(pt_prescs['eventdate']<(index_date-timedelta(days=365)*5))]

In [64]:
relev_pt_prescs

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq,route,substance strength,drug substance name,index_date
1378550,7587013,1997-12-02,1997-12-02,1150125,674,9013,0,9,100.0,0.000,0,0.0,0,2,Inhalation,1mg/1ml,Salbutamol sulfate,2007-11-23
1378551,7587013,1997-12-02,1997-12-02,1150125,3040,9013,8188,32,3.0,0.286,0,0.0,1197,3,Transdermal,50microgram/24hour,Estradiol,2007-11-23
1378552,7587013,1997-12-02,1997-12-02,1150125,3579,9013,13279,82,100.0,0.000,0,0.0,6,3,Oral,5mg/1ml,Thioridazine,2007-11-23
1378553,7587013,1997-12-02,1997-12-02,1150125,19,9013,1409,55,100.0,6.000,0,0.0,0,3,Oral,500mg + 8mg,Paracetamol/Codeine phosphate,2007-11-23
1378554,7587013,1997-12-02,1997-12-02,1150125,3363,9013,107,10,2.0,2.000,0,0.0,1547,4,Inhalation,400microgram,Beclometasone dipropionate,2007-11-23
1378555,7587013,1997-12-02,1997-12-02,1150125,1962,9013,0,213,112.0,0.000,0,0.0,0,4,Nebulised,0.25mg/ml,Ipratropium Bromide,2007-11-23
1378556,7587013,1997-12-02,1997-12-02,1150125,1358,9013,101476,67,20.0,0.000,0,0.0,0,4,Oral,2.5mg + 25microgram,Diphenoxylate hydrochloride/Atropine sulfate,2007-11-23
1378557,7587013,1997-12-02,1997-12-02,1150125,907,9013,0,9,1.0,0.000,0,0.0,0,5,Inhalation,500microgram/1dose,Terbutaline sulfate,2007-11-23
1378558,7587013,1997-12-02,1997-12-02,1150125,162,9013,19793,4,60.0,2.000,0,0.0,0,5,Oral,50mg + 200microgram,Diclofenac sodium/Misoprostol,2007-11-23
1378559,7587013,1997-12-02,1997-12-02,1150125,30,9013,22,39,56.0,1.000,0,0.0,0,5,Oral,15mg,Lansoprazole,2007-11-23


In [63]:
set(relev_pt_prescs['drug substance name'])

{'Aluminium hydroxide / Magnesium carbonate co-gel/Magnesium alginate/Potassium bicarbonate/Magnesium carbonate',
 'Amoxicillin trihydrate',
 'Amoxicillin trihydrate/Potassium clavulanate',
 'Beclometasone dipropionate',
 'Citric acid monohydrate',
 'Diazepam',
 'Diclofenac sodium/Misoprostol',
 'Dihydrocodeine tartrate',
 'Diphenoxylate hydrochloride/Atropine sulfate',
 'Estradiol',
 'Fluoxetine hydrochloride',
 'Furosemide',
 'Glyceryl Trinitrate',
 'Ibuprofen',
 'Ipratropium Bromide',
 'Lansoprazole',
 'Loratadine',
 'Lorazepam',
 'Oxybutynin hydrochloride',
 'Oxytetracycline dihydrate',
 'Paracetamol/Codeine phosphate',
 'Paroxetine hydrochloride',
 'Prednisolone',
 'Promethazine hydrochloride',
 'Propiverine hydrochloride',
 'Risperidone',
 'Salbutamol sulfate',
 'Sodium alginate/Calcium carbonate/Sodium bicarbonate',
 'Sodium valproate',
 'Terbutaline sulfate',
 'Thioridazine',
 'Venlafaxine hydrochloride'}

In [30]:
relev_pt_prescs.loc[relev_pt_prescs['drug substance name'].str.contains('zolpidem|zopiclone',case=False,na=False)].sort_values(by='eventdate')

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq,route,substance strength,drug substance name,index_date


In [61]:
# pegmed = pd.read_csv('data/dicts/proc_pegasus_medical.csv',delimiter=',')

In [213]:
pegmed[pegmed['medcode']==11788.0]

,medcode,readcode,clinical events,immunisation events,referral events,test events,read term,database build
35842,11788,1371.11,737374,0,12,1,Non-smoker,February 2009
